# Part 2 : Dyna-Q and Experience replay

The paper by Lin et. al. [1], addresses the issue of sluggish leaning by the reinforcement learning (RL) frameworks. The primary frameworks in question are *adaptive heuristic critic* (AHC) and *Q-learning*. To speed up these two frameworks the paper proposes three extensions to these algorithms. In total they implement and compare 8 different frameworks:
 - Vanilla: AHCON and QCON
 - Experience Replay: AHCON-R and QCON-R
 - Action modeling (inspired by Dyna architechture by Sutton et. al.[2]) : AHCON-M and QCON-M
 - Teaching: AHCON-T and QCON-T
 
 In this report we will be comparing the QCON-R and QCON-M with the vanilla Dyna-Q framework proposed in [2] and as implemented earlier in this report. We choose QCON-R and QCON-M since the these use are based on estimating action-value function $Q(S,A)$ just like the Dyna-Q framework, even though how they achieve this is very diffrent compared to Dyna-Q Each of these methods also have it's own implementation of storing and reusing historical experiences of the agent. Hence it's most reasonable to compare these with Dyna-Q.

### Tabular Dyna-Q

- Initialize $Q(s, a)$ and $Model(s, a)$ for all $s \in S$ and $a \in A(s)$ 
- Loop forever or until $S \in S(terminal)$:
  - $S \leftarrow state_{current}$
  - $A \leftarrow \epsilon - greedy(S,Q)$
  - $\acute{S}, R \leftarrow environment(S,A)$
  - $Q(S,A) \leftarrow Q(S,A) + \alpha[ R + \gamma\ max_{a}(\ Q(\acute{S}, a)\ ) − Q(S,A)]$
  - $model(S,A) \leftarrow R,\acute{S} $
  - Loop repeat $n$ times:
      - $S \leftarrow$ random previously observed state
      - $A \leftarrow$ random action previously taken in $S$
      - $R,\acute{S} \leftarrow model(S,A) $
      - $Q(S,A) \leftarrow Q(S,A) + \alpha[ R + \gamma\ max_{a}(\ Q(\acute{S}, a)\ ) − Q(S,A)]$

This is the framework presented by Sutton et. al. [2]. We implement this earlier in this report on two test *dynamic maze game worlds*. The idea behind the framework is to unify the *learning* and *planning* arms of the RL Learning. These are also known as *model free* and *model based* reinforcement learning. The following two approches (QCON-M and QCON-R) also address this unification. The general idea of the unification among all the three algorithms is the same: 

1. Get real world experience
2. Learning: Update value function
3. Simulate experiences from History
4. Update value function using these simulated experiences
5. Repeat

The key between Dyna-Q and others is how they implement the step 2, 3 and 4. The action selection here also differs from the other two algorithms where they use a softmax function over the q-value (or utility function as the paper calls it). Here we use a simple $\epsilon-greedy$ algorithm. For learning we implement the tabular version of Q-learning algorithm as our learning function. As for history, it is stored by maintaining a oracle. In particular we maintain a lookup table of the form:

$model(S, A) = (\acute{S}, R)$

The model is first initialised such that

$model(S, a) = (S, 0), a \in A$

And at each step the lookup table is deterministicall updated to reflect the latest experience. This way we lose the past observations experienced at by taking action $A$ while in state $S$. This oracle is then used to randomly generate simulated experiences. These are chosen only from the experiences which have to been experienced in the real world in the past. We then use these experiences to further update the value function (Q-learning update in our case). We do this since it is usually very expensive to collect real world data than to generate simulations. This way we can bootstrap our learning on past data with little cost.

### QCON-M

- Initialize $Q(s, a)$ and $Model(s, a)$ for all $s \in S$ and $a \in A(s)$ 
- Loop forever or until $S \in S(terminal)$:
  1. $S \leftarrow state_{current}$
  2. $A \leftarrow softmax(S,Q_{net})$
  3. $\acute{S}, R \leftarrow environment(S,A)$
  4. $\Delta Q \leftarrow [R + \gamma\ Q_{net}(\acute{S},A)] - Q_{net}(S,A)$
  5. update $Q_{net}$ by progating $\Delta Q$ through the network
  6. $model(S,A) \leftarrow R,\acute{S} $
  7. Loop repeat $n$ times:
      - $S \leftarrow$ random previously observed state $S$ from $model \ s.t. \ softmax(S_{current},Q)_{S} \ge threshold$
      - $A \leftarrow$ random action previously taken in $S$
      - $R,\acute{S} \leftarrow model(S,A) $
      - $\Delta Q \leftarrow [R + \gamma\ Q_{net}(\acute{S},A)] - Q_{net}(S,A)$
      - update $Q_{net}$ by progating $\Delta Q$ through the network

QCON-M is inspired by the Dyna-Q architechture, explained above, with some differences. Apart from the previously mentioned action selection (policy) main difference is that the action-value function here is estimated by using a neural network instead of the tabular Q-learning update. Because of this, the method of simulating past experiences defers from Dyna-Q. Since a Q-value update in the nueral network would affect the network as a whole, we do not want a experiences of a comparatively bad policy (experienced in the past) to affect the current value, induced by the current policy. This problem does not arise in the case of tabular updates since we do the update only for a particular pair of state and action. Hence only those experiences from the past are replayed which also occur under the current policy of the algorithm, **given the current state**. This is another subtle difference of this framework from the Dyna-Q presented above, where simulations are chosen starting from any random state. Since the value-network, given a state $S$ and action $A$ produces a probability distribution over the the next possible states $\acute{S}$, the paper uses a threshold over these probabilities for selecting past experiences. Model updates are carried out in the exact same way as in the case of Dyna-Q

### QCON-R

- Initialize $Q(s, a)$ and $Model(s, a)$ for all $s \in S$ and $a \in A(s)$ 
- Loop forever or until $S \in S(terminal)$:
  1. $S \leftarrow state_{current}$
  2. $A \leftarrow softmax(S,Q)$
  3. $\acute{S}, R \leftarrow environment(S,A)$
  4. $\Delta Q \leftarrow [R + \gamma\ Q_{net}(\acute{S},A)] - Q_{net}(S,A)$
  5. update $Q_{net}$ by progating $\Delta Q$ through the network
  6. $H_t \leftarrow (S, A, R,\acute{S}) $
  7. Loop repeat $n$ times:
      - $S \leftarrow$ random previously observed state $S$ from $H \ s.t. \ softmax(S_{current},Q)_{S} \ge threshold$
      - $A \leftarrow$ random action previously taken in $S$
      - $R,\acute{S} \leftarrow model(S,A) $
      - $\Delta Q \leftarrow [R + \gamma\ Q_{net}(\acute{S},A)] - Q_{net}(S,A)$
      - update $Q_{net}$ by progating $\Delta Q$ through the network

QCON-R is similar to the QCON-M since the mechanism of value network and action selection remains the same. But the this framework does not maintain a model of past experiences. Instead it simply records all of the history occured till now. But similar to QCON-M, only those past experiences which satisfy the current policy are selected for simulation.

## References

1. Lin, L.J., 1992. Self-improving reactive agents based on reinforcement learning, planning and teaching. Machine learning, 8(3-4), pp.293-321.
2. Sutton, R.S. and Barto, A.G., 1998. Reinforcement learning: An introduction (Vol. 1, No. 1). Cambridge: MIT press.